# 원, 삼각형, 사각형 구분하기
## 3. Data Augmentation을 사용한 CNN 모델

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# seed 값 설정
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 부풀리기(Augmentation)

In [3]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [4]:
# 데이터 부풀리기를 적용한 훈련 데이터 제너레이터
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.8, 2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [5]:
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

# 무한 for loop이니 주의!!!
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview', 
                                save_prefix='tri', save_format='png'):
    i += 1
    if i > 30: 
        break

### 데이터셋 생성하기

In [6]:
train_generator = train_datagen.flow_from_directory(
        'hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


### 모델 구성하기

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

### 모델 학습과정 설정하기

In [8]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

### 모델 학습시키기
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 200번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 validation_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [9]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15*100,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/200
1500/1500 [==============================] - 23s 15ms/step - loss: 0.1511 - accuracy: 0.9411 - val_loss: 3.7163e-04 - val_accuracy: 0.7333
Epoch 2/200
1500/1500 [==============================] - 28s 19ms/step - loss: 0.0355 - accuracy: 0.9889 - val_loss: 3.6298 - val_accuracy: 0.6667
Epoch 3/200
1500/1500 [==============================] - 29s 19ms/step - loss: 0.0279 - accuracy: 0.9900 - val_loss: 0.9012 - val_accuracy: 0.6667
Epoch 4/200
1500/1500 [==============================] - 29s 19ms/step - loss: 0.0232 - accuracy: 0.9922 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 5/200
1500/1500 [==============================] - 29s 19ms/step - loss: 0.0136 - accuracy: 0.9964 - val_loss: 9.7473 - val_accuracy: 0.7333
Epoch 6/200
1500/1500 [==============================] - 27s 18ms/step - loss: 0.0109 - accuracy: 0.9967 - val_loss: 0.7037 - val_accuracy: 0.8000
Epoch 7/200
1500/1500 [==============================] - 26s 17ms/step - loss: 0.0113 - accuracy: 0.9971 - val

1500/1500 [==============================] - 29s 19ms/step - loss: 1.8068e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 110/200
1500/1500 [==============================] - 29s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 11.1197 - val_accuracy: 0.8000
Epoch 111/200
1500/1500 [==============================] - 29s 19ms/step - loss: 1.5895e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 112/200
1500/1500 [==============================] - 27s 18ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 94.4987 - val_accuracy: 0.8000
Epoch 113/200
1500/1500 [==============================] - 29s 19ms/step - loss: 1.0596e-10 - accuracy: 1.0000 - val_loss: 39.7018 - val_accuracy: 0.8000
Epoch 114/200
1500/1500 [==============================] - 27s 18ms/step - loss: 5.2982e-11 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 0.8000
Epoch 115/200
1500/1500 [==============================] - 29s 19ms/step - loss: 

### 모델 평가하기

In [10]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
accuracy: 80.00%


### 모델 사용하기

In [11]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'circle': 0, 'rectangle': 1, 'triangle': 2}
[[0.000 1.000 0.000]
 [0.000 0.000 1.000]
 [0.000 1.000 0.000]
 [0.000 1.000 0.000]
 [1.000 0.000 0.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [1.000 0.000 0.000]
 [1.000 0.000 0.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.233 0.000 0.767]
 [0.000 1.000 0.000]]


In [12]:
for i in range(len(test_generator.labels)):
    print(test_generator.filenames[i], test_generator.labels[i], np.argmax(output[i]))

circle\circle021.png 0 1
circle\circle022.png 0 2
circle\circle023.png 0 1
circle\circle024.png 0 1
circle\circle025.png 0 0
rectangle\rectangle021.png 1 2
rectangle\rectangle022.png 1 2
rectangle\rectangle023.png 1 2
rectangle\rectangle024.png 1 0
rectangle\rectangle025.png 1 0
triangle\triangle021.png 2 2
triangle\triangle022.png 2 2
triangle\triangle023.png 2 2
triangle\triangle024.png 2 2
triangle\triangle025.png 2 1
